In [1]:
import torch

a = torch.randn(3, 4, requires_grad=True)
b = torch.randn(3, 4, requires_grad=True)
print(a)
print(b)
x = a + b
print(x.shape)
print(x)
y = x.sum()
print(y)
print("-" * 50)
y.backward()
print(b.grad)

tensor([[-0.0715, -1.8414, -1.3939, -0.1334],
        [ 0.2784, -0.0738,  0.3051,  0.7567],
        [ 1.0804, -1.0013, -2.6729, -0.7928]], requires_grad=True)
tensor([[-0.3077,  0.2291,  0.7008, -1.1828],
        [ 0.7223,  1.8537, -0.2271,  2.3113],
        [ 1.2290, -0.7723,  0.7342, -0.6633]], requires_grad=True)
torch.Size([3, 4])
tensor([[-0.3792, -1.6123, -0.6931, -1.3162],
        [ 1.0008,  1.7799,  0.0780,  3.0680],
        [ 2.3093, -1.7736, -1.9387, -1.4561]], grad_fn=<AddBackward0>)
tensor(-0.9331, grad_fn=<SumBackward0>)
--------------------------------------------------
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


In [18]:
import torch

a = torch.rand(1)
b = torch.rand(1, requires_grad=True)
c = torch.rand(1, requires_grad=True)
print(a, b, c)
x = c * a
y = x + b
print(a.requires_grad, b.requires_grad, c.requires_grad, x.requires_grad, y.requires_grad)
print(a.is_leaf, b.is_leaf, c.is_leaf, x.is_leaf, y.is_leaf)  # 叶子

tensor([0.1744]) tensor([0.3201], requires_grad=True) tensor([0.0203], requires_grad=True)
False True True True True
True True True False False
tensor([0.1744]) tensor([1.])


In [21]:
y.backward(retain_graph=True)  # 如果不清空会累加起来
print(c.grad, b.grad)

tensor([0.6975]) tensor([4.])


In [39]:
import numpy as np

x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)
print(x_train.shape)
y_values = [2 * i + 1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
print(y_train.shape)

(11, 1)
(11, 1)


In [52]:
# 线性回归模型
import torch


class LinearRegressionModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out


input_dim = 1
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)
model

cuda


LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [43]:
# 指定参数和损失函数
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()

In [48]:
# 训练模型
for epoch in range(epochs):
    epoch += 1
    inputs = torch.from_numpy(x_train).to(device)
    labels = torch.from_numpy(y_train).to(device)
    # 梯度要清零
    optimizer.zero_grad()
    # 前向传播
    output = model(inputs)
    # 计算损失
    loss = criterion(output, labels)
    # 反向传播
    loss.backward()
    # 更新权重参数
    optimizer.step()
    if epoch % 50 == 0:
        print('Epoch %d/%d, Loss: %.4f' % (epoch, epochs, loss.item()))


Epoch 50/1000, Loss: 61.9269
Epoch 100/1000, Loss: 61.9269
Epoch 150/1000, Loss: 61.9269
Epoch 200/1000, Loss: 61.9269
Epoch 250/1000, Loss: 61.9269
Epoch 300/1000, Loss: 61.9269
Epoch 350/1000, Loss: 61.9269
Epoch 400/1000, Loss: 61.9269
Epoch 450/1000, Loss: 61.9269
Epoch 500/1000, Loss: 61.9269
Epoch 550/1000, Loss: 61.9269
Epoch 600/1000, Loss: 61.9269
Epoch 650/1000, Loss: 61.9269
Epoch 700/1000, Loss: 61.9269
Epoch 750/1000, Loss: 61.9269
Epoch 800/1000, Loss: 61.9269
Epoch 850/1000, Loss: 61.9269
Epoch 900/1000, Loss: 61.9269
Epoch 950/1000, Loss: 61.9269
Epoch 1000/1000, Loss: 61.9269


In [60]:
# 模型预测
predicted = model(torch.from_numpy(x_train).to(device).requires_grad_()).data
predicted

tensor([[ -0.9258],
        [ -1.8744],
        [ -2.8229],
        [ -3.7715],
        [ -4.7200],
        [ -5.6686],
        [ -6.6171],
        [ -7.5656],
        [ -8.5142],
        [ -9.4627],
        [-10.4113]], device='cuda:0')

In [ ]:
# 模型保存和读取
torch.save(model.state_dict(), 'model.pth')
model.load_state_dict(torch.load('model.pth'))